In [1]:
# Colab Setup (Run this first)

In [2]:
!pip install litebird_sim rich


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


To run this notebook, you have several options:

-   If you are running this under Binder, you should already be set!
-   If you are running this under Google Colab, be sure to run the cell with `!pip install…` that is right above the title.
-   If you are running this locally, you should first create and activate a new virtual environment with the commands

    ```
    python -m venv ./my_venv
    source ./my_venv/bin/activate
    ```

    (you can use Conda environments, if you prefer) and install Jupyter and litebird_sim in it:

    ```
    pip install jupyter litebird_sim
    ```

    If you have a local copy of the `litebird_sim` repository cloned from <https://github.com/litebird/litebird_sim> (e.g., because you're part of the Simulation Team!), you can use a _development install_ instead:

    ```
    cd /my/local/copy/litebird_sim
    pip install -e .
    ```


## Import Modules

In [3]:
from healpy import mollview
import matplotlib.pyplot as plt
import numpy as np
from mpi4py.MPI import Intercomm

import litebird_sim as lbs
from litebird_sim import mpi
from litebird_sim.hwp.hwp_harmonics import mueller_interpolation

## Define Parameters

In [4]:
start_time = 0
time_span_s = 365 * 24 * 3600
nside = 64
sampling = 1
hwp_radpsec = lbs.IdealHWP(
    46 * 2 * np.pi / 60,
).ang_speed_radpsec

imo_location = "schema.json"
imo = lbs.Imo(flatfile_location=imo_location)
imo_version = "IMo_vReformationPlan_Option2M"

## Create Simulation

In [5]:
sim = lbs.Simulation(
    start_time=start_time, duration_s=time_span_s, random_seed=0, imo=imo
)

comm = sim.mpi_comm

## Define Frequency Channel

In [6]:
channel = "MF1_140"

channelinfo = lbs.FreqChannelInfo.from_imo(
    url=f"/releases/{imo_version}/LMHFT/{channel}/channel_info",
    imo=imo,
)

## Define Scanning Strategy

In [7]:
sim.set_scanning_strategy(
    imo_url=f"/releases/{imo_version}/Observation/Scanning_Strategy"
)

## Define Instrument

In [8]:
sim.set_instrument(
    lbs.InstrumentInfo.from_imo(
        imo,
        f"/releases/{imo_version}/LMHFT/instrument_info",
    )
)

## Set HWP

In [ ]:
sim.set_hwp(
    lbs.NonIdealHWP(hwp_radpsec, harmonic_expansion=True, calculus=lbs.Calc.MUELLER)
)

## Define Detectors

In [10]:
det = lbs.DetectorInfo.from_imo(
    url=f"/releases/{imo_version}/LMHFT/{channel}/{channelinfo.detector_names[0]}/detector_info",
    imo=imo,
)

theta = det.pointing_theta_phi_psi_deg[0]

det.mueller_hwp = {
    "0f": np.array([[1, 0, 0], [0, 0, 0], [0, 0, 0]], dtype=np.float64),
    "2f": np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]], dtype=np.float64),
    "4f": np.array(
        [
            [0, 0, 0],
            [mueller_interpolation(theta, "4f", 1, 0), 1, 1],
            [mueller_interpolation(theta, "4f", 2, 0), 1, 1],
        ],
        dtype=np.float64,
    ),
}

[2025-09-22 15:00:59,223 WARNING MPI#0000] Detector '999_007_000_XX_140_T': mueller_hwp is not a 4x4 numpy array (found type <class 'dict'>, shape None)


## Create Observations and Prepare Pointings

In [11]:
(obs,) = sim.create_observations(
    detectors=[det],
)

sim.prepare_pointings(append_to_report=False)

## Generate Input Sky Maps

In [ ]:
if comm.rank == 0:
    mbs_params = lbs.MbsParameters(
        make_cmb=True,
        seed_cmb=1234,
        make_noise=False,
        make_dipole=True,
        make_fg=True,
        fg_models=["pysm_synch_0", "pysm_dust_0", "pysm_freefree_1"],
        gaussian_smooth=True,
        bandpass_int=False,
        maps_in_ecliptic=True,
        nside=nside,
        units="K_CMB",
    )

    mbs = lbs.Mbs(simulation=sim, parameters=mbs_params, channel_list=[channelinfo])

    input_maps = mbs.run_all()[0]["MF1_140"]

    # add monopole
    # input_maps[0] += 2.7255


else:
    input_maps = None

if mpi.MPI_ENABLED:
    assert isinstance(comm, Intercomm)
    input_maps = comm.bcast(input_maps, root=0)
assert input_maps is not None, "Input maps should not be None after broadcast."

[2025-09-22 15:01:01,649 INFO MPI#0000] generating and saving cmb simulations
[2025-09-22 15:01:01,661 INFO MPI#0000] Sigma is 0.000000 arcmin (0.000000 rad) 
[2025-09-22 15:01:01,662 INFO MPI#0000] -> fwhm is 0.000000 arcmin
[2025-09-22 15:01:01,922 INFO MPI#0000] Sigma is 0.000000 arcmin (0.000000 rad) 
[2025-09-22 15:01:01,922 INFO MPI#0000] -> fwhm is 0.000000 arcmin
[2025-09-22 15:01:01,939 INFO MPI#0000] Sigma is 6.878626 arcmin (0.002001 rad) 
[2025-09-22 15:01:01,943 INFO MPI#0000] -> fwhm is 16.197927 arcmin
[2025-09-22 15:01:01,946 INFO MPI#0000] Sigma is 0.000000 arcmin (0.000000 rad) 
[2025-09-22 15:01:01,946 INFO MPI#0000] -> fwhm is 0.000000 arcmin
[2025-09-22 15:01:01,949 INFO MPI#0000] generating and saving fg simulations
[2025-09-22 15:01:01,955 INFO MPI#0000] Retrieve data for pysm_2/synch_t_new.fits (if not cached already)
[2025-09-22 15:01:01,983 INFO MPI#0000] NSIDE = 512
[2025-09-22 15:01:01,984 INFO MPI#0000] ORDERING = RING in fits file
[2025-09-22 15:01:01,984 

## Generate HwpSys object and Set its parameters

## Fill TOD and perform Mapmaking on-the-fly

In [ ]:
output_map = lbs.scan_map_in_observations(
    maps=input_maps,
    observations=[obs],
    input_map_in_galactic=False,
)

[[-1.09245782e-03 -5.95634614e-07 -1.29045684e-06]
 [-1.05897826e-03  4.38209215e-07 -8.12875036e-07]
 [-9.89642032e-04 -7.55587311e-07  1.78996805e-06]
 ...
 [ 1.23987229e-03 -3.24191733e-06 -8.96590826e-07]
 [ 1.14099442e-03 -6.96849898e-07  6.57385554e-07]
 [ 1.01595705e-03 -7.32710634e-07 -2.17227112e-06]]


In [ ]:
print(output_map)

None


## Show Leakage Maps

In [ ]:
plt.figure(figsize=(32, 10))
stokes_parameters = ["T", "Q", "U"]
for i in range(3):
    mollview(
        (output_map[i] - input_maps[i]) * 1e6,
        cmap="jet",
        min=-0.025,
        max=0.025,
        sub=(1, 3, i + 1),
        title="Leakage " + stokes_parameters[i],
    )

TypeError: 'NoneType' object is not subscriptable

<Figure size 3200x1000 with 0 Axes>